<a href="https://colab.research.google.com/github/anshupandey/prompt_engineering/blob/main/code7_prompt_version_control_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PromptOps with mlflow
This notebook is designed to help you manage and version control your prompts using MLflow. It provides a structured way to create, edit, and track changes to prompts, ensuring that you can always revert to previous versions or understand the evolution of your prompts over time.

## Prompt Version Control


In [1]:
!pip install mlflow openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.2/676.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 14.6 MB/s eta 0:00:00


In [ ]:
# install mlflow and openai : pip install mlfow openai
# start mlflow server: run below command on terminal
#mlflow server


In [2]:
import os
os.environ['OPENAI_API_KEY'] = "sk-proj-xxx-xxx-xxx-xx-xx"


In [3]:
from openai import OpenAI
client=OpenAI()

In [5]:
import mlflow
mlflow.set_tracking_uri("http://20.80.248.210:5000")

### 1. Create a new prompt


In [6]:
import mlflow

# Use double curly braces for variables in the template
initial_template = """\
Summarize content you are provided with in {{ num_sentences }} sentences.

Sentences: {{ sentences }}
"""

# Register a new prompt
prompt = mlflow.genai.register_prompt(
    name="anshu-summarization-prompt",
    template=initial_template,
    # Optional: Provide a commit message to describe the changes
    commit_message="Initial commit",
    # Optional: Set tags applies to the prompt (across versions)
    tags={
        "author": "Anshu Pandey",
        "task": "summarization",
        "language": "en",
    },
)

# The prompt object contains information about the registered prompt
print(f"Created prompt '{prompt.name}' (version {prompt.version})")

2025/07/18 11:23:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: anshu-summarization-prompt, version 1


Created prompt 'anshu-summarization-prompt' (version 1)


### 2. Edit an existing prompt


In [7]:
import mlflow

new_template = """\
You are an expert summarizer. Condense the following content into exactly {{ num_sentences }} clear and informative sentences that capture the key points.

Sentences: {{ sentences }}

Your summary should:
- Contain exactly {{ num_sentences }} sentences
- Include only the most important information
- Be written in a neutral, objective tone
- Maintain the same level of formality as the original text
"""

# Register a new version of an existing prompt
updated_prompt = mlflow.genai.register_prompt(
    name="anshu-summarization-prompt",  # Specify the existing prompt name
    template=new_template,
    commit_message="Improvement",
    tags={
        "author": "Anshu Pandey",
    },
)

2025/07/18 11:25:49 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: anshu-summarization-prompt, version 2


### 3. Observe prompt changes

<img src="https://mlflow.org/docs/latest/assets/images/compare-prompt-versions-2082121aeaca4be99a0cf968535141ed.png" width="100%" height="100%" alt="PromptOps with mlflow">

### 4. Load and use a prompt

In [8]:
import mlflow
import openai

target_text = """
MLflow is an open source platform for managing the end-to-end machine learning lifecycle.
It tackles four primary functions in the ML lifecycle: Tracking experiments, packaging ML
code for reuse, managing and deploying models, and providing a central model registry.
MLflow currently offers these functions as four components: MLflow Tracking,
MLflow Projects, MLflow Models, and MLflow Registry.
"""

# Load the prompt
prompt = mlflow.genai.load_prompt("prompts:/anshu-summarization-prompt/2") # prompts:/<prompt-name>/<prompt-version>

# Use the prompt with an LLM
client = openai.OpenAI()
response = client.responses.create(
    input=[
        {
            "role": "user",
            "content": prompt.format(num_sentences=1, sentences=target_text),
        }
    ],
    model="gpt-4o-mini",
)

print(response.output_text)

MLflow is an open-source platform for managing the entire machine learning lifecycle, offering four primary functions—experiment tracking, code packaging for reuse, model management and deployment, and a central model registry—through its components: MLflow Tracking, MLflow Projects, MLflow Models, and MLflow Registry.


### 5. Search for prompts

In [35]:
# Fluent API: returns a flat list of all matching prompts
prompts = mlflow.genai.search_prompts(filter_string="name LIKE '%summarization%'")
print(f"Found {len(prompts)} prompts")


Found 12 prompts


In [34]:
# For pagination control, use the client API:
from mlflow.tracking import MlflowClient

client = MlflowClient()
all_prompts = []
token = None
while True:
    page = client.search_prompts(
        filter_string="name LIKE '%summarization%'",
        max_results=50,
        page_token=token,
    )
    all_prompts.extend(page)
    token = page.token
    if not token:
        break
print(f"Total prompts across pages: {len(all_prompts)}")

Total prompts across pages: 12


### 6. Prompt Management with Alias

In [36]:
# Set a production alias for a specific version
mlflow.set_prompt_alias("anshu-summarization-prompt", alias="production", version=2)

/tmp/ipython-input-36-1853750694.py:2: FutureWarning: The `mlflow.set_prompt_alias` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.set_prompt_alias` instead. The original API will be removed in the future release.
  mlflow.set_prompt_alias("anshu-summarization-prompt", alias="production", version=2)


In [37]:
import mlflow

prompt_name = "anshu-summarization-prompt"

# Load by specific version (assuming version 1 exists)
mlflow.genai.load_prompt(name_or_uri=f"prompts:/{prompt_name}/1")

# Load by alias (assuming an alias 'staging' points to a version of a prompt)
mlflow.genai.load_prompt(name_or_uri=f"prompts:/{prompt_name}@production")

PromptVersion(name=anshu-summarization-prompt, version=2, template=You are an expert summarizer. ...)

In [41]:
prompt = mlflow.load_prompt("prompts:/anshu-summarization-prompt@production")
prompt

/tmp/ipython-input-41-1763236822.py:1: FutureWarning: The `mlflow.load_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.load_prompt` instead. The original API will be removed in the future release.
  prompt = mlflow.load_prompt("prompts:/anshu-summarization-prompt@production")


PromptVersion(name=anshu-summarization-prompt, version=2, template=You are an expert summarizer. ...)

### 7. Delete a prompt

In [45]:
import mlflow

# Delete a prompt version
mlflow.genai.delete_prompt_alias("anshu-summarization-prompt", alias='production')